In [72]:
import pandas as pd
import operator as op
f = open('dna.example.fasta')
fasta = f.read()
record = pd.DataFrame()
fasta_id = fasta.split('>') #splice into sequences
fasta_id
del fasta_id[0] #delate first item, which's ''
fasta_id[0].split('\n')[0].split(',')[0]
len(''.join(fasta_id[0].splitlines()[1:-1]))

980

In [76]:
#
Sequence_identifier = []
Sequence_description = []
Sequence = []
Seq_len = [] 
for sequence in fasta_id:
    lines = sequence.splitlines() #separate each line, first line(item) is the ID&description
    Sequence_identifier.append(lines[0].split(',')[0]) #separate by ',' first item is ID
    Sequence_description.append(lines[0].split(',')[1]) #second item is description
    Sequence.append( ''.join(lines[1:-1])) #join the rest items in the list as the sequence
    Seq_len.append(len(''.join(lines[1:-1])))
record['Seq_id'] = Sequence_identifier
record['Seq_description'] = Sequence_description
record['Sequence'] = Sequence
record['Sequence_length'] = Seq_len
record.head(8)

,Seq_id,Seq_description,Sequence,Sequence_length
0,gi|142022655|gb|EQ086233.1|43 marine metagenom...,whole genome shotgun sequence,TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGC...,980
1,gi|142022655|gb|EQ086233.1|160 marine metageno...,whole genome shotgun sequence,ATTGGGGAGGAGGCGAGTTGAGCGGCGGCAGTTCGCCTGCGTGCGC...,700
2,gi|142022655|gb|EQ086233.1|41 marine metagenom...,whole genome shotgun sequence,GACCTTGCATCGGCTGATCGCCGAGCGTGCCGACGTATTCATTCAC...,3010
3,gi|142022655|gb|EQ086233.1|221 marine metageno...,whole genome shotgun sequence,GCCCGGCGATTTGACGTCTGCAGCCTCACGTCCAAACTCAATTTGA...,2800
4,gi|142022655|gb|EQ086233.1|294 marine metageno...,whole genome shotgun sequence,GATCAGCCCCGCATACGCGTACGCGCGTGCGACGCCGAAGAGCGTC...,3780
5,gi|142022655|gb|EQ086233.1|323 marine metageno...,whole genome shotgun sequence,ACGCCCGGCGCACCGCGAGTACCGCGCCGCCGGGCACTCCTTGACC...,4760
6,gi|142022655|gb|EQ086233.1|564 marine metageno...,whole genome shotgun sequence,CTGGTAGCCATGCAGCAAGGCTGGCGCTAGATGTACGGCCAGATTG...,1610
7,gi|142022655|gb|EQ086233.1|521 marine metageno...,whole genome shotgun sequence,CGTTGTTCGCCAGGTCGTCCGCATAGCCGGCCGAGCTGAACTGCGT...,490


In [77]:
longest_seq = record[record['Sequence_length'] == max(record['Sequence_length'])]['Seq_id'].tolist()[0]#getting ID for the longest sequence
shortest_seq = record[record['Sequence_length'] == min(record['Sequence_length'])]['Seq_id'].tolist()[0]#getting ID for the shortest sequence
longest_seq

'gi|142022655|gb|EQ086233.1|323 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold'

In [122]:
'''This function takes in the sequence and the # of reading frame(1,2 or 3), first it searches for a start condon,
then, it searches for the nearest inframe stop codon, after finding the stop codon, it append the sequence between the 
start codon and stop codon to the ORF list'''
def checkORF(seq, frame): #giving the sequence and readingframe, return all ORF as dict with keys-sequence,value-start position,
                          # of ORF, length of the longest ORF
    ORF = {}
    stop_codon = ['TAA', 'TAG','TGA']
    for i in range(frame,len(seq)-3,3):
        if seq[i:i+3].upper() == 'ATG':
            for j in range(i+3,len(seq)-3,3):
                if seq[j:j+3].upper() in stop_codon:
                    ORF[seq[i:j+3]] = i
                    break
    return ORF, len(ORF) #len(max(ORF, key = len))
       

def totalORF(seq): #total reading frame for all three possibility
    checkORF(seq,1)
    checkORF(seq,2)
    checkORF(seq,3)
    total_number = checkORF(seq,1)[1]+checkORF(seq,2)[1]+checkORF(seq,3)[1]
    all_ORF = dict(list(checkORF(seq,1)[0].items())+list(checkORF(seq,2)[0].items())+list(checkORF(seq,3)[0].items()))
    return all_ORF,total_number

def longestORF(seq): #return the longest ORF,start position,the length
    frame = max((totalORF(seq)[0]),key = len)
    length = len(frame)
    position = totalORF(seq)[0][frame]
    return frame,position,length


#def ORF_start(seq): #return the position of the longest ORF
    
    
#list(checkORF(record['Sequence'][5],3)[0].keys())
#totalORF(record['Sequence'][5])
#longestORF(record['Sequence'][5])
se = record['Sequence'][0]
#totalORF(record['Sequence'][5])
#checkORF(record['Sequence'][5],1)[1]
#longestORF(se)
checkORF(se,3)
#totalORF(se)                                 


({'ATGACAGCTCCGTTCAGGGGGAGGGGGTAA': 549,
  'ATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTTTGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAA': 366,
  'ATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAA': 453,
  'ATGCCCAGCACGCCAATGCGTTCTTCATCCACATAG': 216,
  'ATGCGGCGCTCGGTCATCGCTGCATTGATCGAGTAG': 165,
  'ATGCGTTCTTCATCCACATAG': 231,
  'ATGGCACCTACATGGATCCCTCACTGCTTCCGTCTCTCGCGTGGTTCGCCCACGTCGCACATCATCGTAGCTTCACGAAAGCGGCTGCGGAAATGGGCGTTTCTCGAGCAAACCTGTCGCAGAACGTGA': 690,
  'ATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGCCATAG': 72,
  'ATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAA': 456,
  'ATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAA': 501},
 10)

In [172]:
'''incooperate the ORF info into the dataframe
   - total reading frame --'total_ORF
   - longest reading frame --'longest_ORF'
   - length of the longest reading frame --'longest_ORF_length'
   - start position of longest reading frame -- longest_ORF_position'''
record['total_ORF'] = record['Sequence'].apply(totalORF) 
record['total_ORF'] = record['total_ORF'].apply(op.itemgetter(1)) #total reading frame calculated
record['longest_ORF'] = record['Sequence'].apply(longestORF).apply(op.itemgetter(0))
record['longest_ORF_position'] = record['Sequence'].apply(longestORF).apply(op.itemgetter(1))
record['longest_ORF_length'] = record['longest_ORF'].apply(len)
record['ORF1'] = record.apply(lambda x:checkORF(x['Sequence'],1),axis =1).apply(op.itemgetter(1))
record['longest_ORF1'] = record.apply(lambda x:checkORF(x['Sequence'],1),axis =1).apply(op.itemgetter(0))
len(record['longest_ORF1'])
longest_O1 = []
for item in record['longest_ORF1'].tolist():
    if item != {}:
        longest_O1.append(max(item,key = len))
    else:
        longest_O1.append('NaN')
#longest_O1 = [max(item,key=len) for item in record['longest_ORF1'].tolist() if item !={}]
#len(longest_O1)
record['longest_ORF1'] = longest_O1
record

,Seq_id,Seq_description,Sequence,Sequence_length,total_ORF,longest_ORF,longest_ORF_position,longest_ORF_length,ORF1,longest_ORF1
0,gi|142022655|gb|EQ086233.1|43 marine metagenom...,whole genome shotgun sequence,TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGC...,980,12,ATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCC...,366,213,1,ATGTTTGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGA
1,gi|142022655|gb|EQ086233.1|160 marine metageno...,whole genome shotgun sequence,ATTGGGGAGGAGGCGAGTTGAGCGGCGGCAGTTCGCCTGCGTGCGC...,700,12,ATGCGGGGCGATCAAGATGGGGATGCGGGATGGGGGCGACGGTGTA...,106,363,1,ATGCGGGGCGATCAAGATGGGGATGCGGGATGGGGGCGACGGTGTA...
2,gi|142022655|gb|EQ086233.1|41 marine metagenom...,whole genome shotgun sequence,GACCTTGCATCGGCTGATCGCCGAGCGTGCCGACGTATTCATTCAC...,3010,41,ATGAGTACATTCAACCGATTGCACCTGATTCGTCAGGTGGATTTGT...,1193,918,11,ATGGGCGTTGCGACGCATTCATATCGCCACGGCGCGCGGCCGAGAC...
3,gi|142022655|gb|EQ086233.1|221 marine metageno...,whole genome shotgun sequence,GCCCGGCGATTTGACGTCTGCAGCCTCACGTCCAAACTCAATTTGA...,2800,33,ATGCCTGTTCTTCCGGACTCGCTCTCCTTTCCCGACGCCGCTTCCC...,1771,594,12,ATGCCTGTTCTTCCGGACTCGCTCTCCTTTCCCGACGCCGCTTCCC...
4,gi|142022655|gb|EQ086233.1|294 marine metageno...,whole genome shotgun sequence,GATCAGCCCCGCATACGCGTACGCGCGTGCGACGCCGAAGAGCGTC...,3780,44,ATGCACCGGGCAACCGGCTTCGAACGGACGCCCGCGACGCATCCCG...,140,1608,11,ATGGGTCGCCTGGCGCGTTGCGTCGTCGTGGGCGGCGTCGAGCGTC...
5,gi|142022655|gb|EQ086233.1|323 marine metageno...,whole genome shotgun sequence,ACGCCCGGCGCACCGCGAGTACCGCGCCGCCGGGCACTCCTTGACC...,4760,48,ATGCGAACCAGTTGTTCGGACACCTCGACGAGCAAGTCGTGCCGCA...,2823,1686,20,ATGAAACCCGAAAACCTCGTCGCCTGCCACGAATGCGACCTGCTGT...
6,gi|142022655|gb|EQ086233.1|564 marine metageno...,whole genome shotgun sequence,CTGGTAGCCATGCAGCAAGGCTGGCGCTAGATGTACGGCCAGATTG...,1610,18,ATGGGCCAGCGCCTGCAGCGACTCGCGCATGATCTGGCGGTGCTCG...,824,507,0,NaN
7,gi|142022655|gb|EQ086233.1|521 marine metageno...,whole genome shotgun sequence,CGTTGTTCGCCAGGTCGTCCGCATAGCCGGCCGAGCTGAACTGCGT...,490,3,ATGTACTGCAGCGCCGCCGCGAACGCGGCGTCCGGCGCGTCGAGCA...,125,159,0,NaN
8,gi|142022655|gb|EQ086233.1|455 marine metageno...,whole genome shotgun sequence,TTCGAGTCGCTCGACGCGCCTCGACGGGCAACTAGCCGCCGTTGGC...,630,2,ATGTCGGCCGGGGCTTCGAGCGGCGTCGCGTGTGCTGCAAGCGCCC...,152,54,0,NaN
9,gi|142022655|gb|EQ086233.1|229 marine metageno...,whole genome shotgun sequence,GGGTTCGGCGTCCTACTGGGGCGGCTACTGCAACCACGGCAACGTG...,3010,35,ATGCTGCCGTTCTGGGACGAAACCAGCGAGGAGTCGCTGACGAAAG...,124,1311,8,ATGCTGCCGTTCTGGGACGAAACCAGCGAGGAGTCGCTGACGAAAG...


In [176]:
''' what is the length of the longest ORF in the file?
   What is the identifier of the sequence containing the longest ORF? 
   For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? 
   What is the starting position of the longest ORF in the sequence that contains it? '''
len(record.index)


25

In [200]:
#identify repeats
def findrepeats(n):
    segments = {}
    seg = []
    #record = data.getRecord()
    for item in record['Sequence']:
        for i in range(0,len(item)-n):
            if item[i:i+n] not in seg:
                seg.append(item[i:i+n])
                segments[item[i:i+n]] = 0
            else:
                segments[item[i:i+n]]+=1
    highest_freq = max(segments.values())
    highest_freq_repeats = []
    if highest_freq > 0:       
        for i,j in segments.items():
            if j == highest_freq:
                highest_freq_repeats.append(i)
    else:
        highest_freq_repeats.append('No Repeats Identified')
            
    return highest_freq, highest_freq_repeats
findrepeats(10)

(7, ['GCGCGGCGCG'])

In [185]:
item = 'TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGG'
i = 0
n = 4
item[i:n]

'TCGG'